# Command (polecenie)

# Przykład: ScreenCommand

In [ ]:
class Screen(object):
    def __init__(self, text=''):
        self._text = text
        self._clipboard = ''
        
    def cut(self, start=0, end=0):
        self._clipboard = self._text[start:end]
        self._text = self._text[:start] + self._text[end:]
        
    def paste(self, offset=0):
        self._text = self._text[:offset] + self._clipboard + self._text[offset:]
        
    def clear_clipboard(self):
        self.clipboard = ''
        
    def __len__(self):
        return len(self.text)
    
    def __repr__(self):
        return 'Screen({})'.format(repr(self._text))
    

In [ ]:
class ScreenCommand(object):
    def __init__(self, screen):
        self._screen = screen
        
    def execute(self):
        self._previous_state = screen._text
        
    def undo(self):
        raise NotImplementedError
        
class CutCommand(ScreenCommand):
    def __init__(self, screen, start=0, end=0):
        super().__init__(screen)
        self._start = start
        self._end = end
        
    def execute(self):
        super().execute()
        self._screen.cut(start=self._start, end=self._end)
        
    def undo(self):
        self._screen.clear_clipboard()
        self._screen._text = self._previous_state
        
class PasteCommand(ScreenCommand):
    def __init__(self, screen, offset=0):
        super().__init__(screen)
        self._offset = offset
        
    def execute(self):
        super().execute()
        self._screen.paste(offset=self._offset)
        
    def undo(self):
        self._screen.clear_clipboard()
        self._screen._text = self._previous_state

In [25]:
class ScreenInvoker(object):
    def __init__(self):
        self._history = []
        
    def store_and_execute(self, command):
        command.execute()
        self._history.append(command)
        
    def undo_last(self):
        self._history.pop().undo()

### Oczekiwane zachowanie

In [26]:
screen = Screen('hello world')
screen

Screen('hello world')

In [27]:
cut_command = CutCommand(screen, start=5, end=11)
client = ScreenInvoker()

In [28]:
client.store_and_execute(cut_command)
screen

Screen('hello')

In [29]:
paste = PasteCommand(screen, offset=0)
client.store_and_execute(paste)
screen

Screen(' worldhello')

In [30]:
client.undo_last()
screen

Screen('hello')

In [31]:
client.undo_last()
screen

Screen('hello world')